In [0]:
import sklearn
sklearn.__version__
import scipy
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,r2_score,explained_variance_score
from graphviz import Digraph
# import scipy.stats.f

In [0]:
df = pd.read_csv("synth_data_with_header.csv", header='infer')
# n: number of observations
n = df.shape[0]
k = df.shape[1]
print(n)
print(k)


999
4


In [0]:
def regression(df, x_name, y_name, maxlag):
    data = df
    df_list = []     
    
    # add lagged columns of current x variable as x_name
    for lag in range(1,maxlag+1):
        data['{}_{}'.format(x_name,str(lag))] = data['{}'.format(x_name)].shift(lag)
        df_list.append(data['{}_{}'.format(x_name,str(lag))])

    # create test dataframe X, and y
    X = pd.concat(df_list,axis = 1)  
    y = data[y_name]

    # remove NaN rows, the number of removal is maxlag
    X = X.iloc[maxlag:]
    y = y.iloc[maxlag:]

    # build regression reg_y, X→y
    reg_y = LinearRegression()
    # fit model using data X, y
    reg_y.fit(X, y)
    # check number of features
#     print(reg_y.n_features_)
    # check feature importance 
#     print(reg_y.feature_importances_)

    # y_hat is the predicted value of y
    y_hat = reg_y.predict(X) 

    # save predicted y_hat as a pd dataframe and move its index to match the place in original df
    y_hat_df = pd.DataFrame(y_hat)
    y_hat_df.index += maxlag
    # save the predicted value into dataframe
    data['predicted_{}'.format(y_name)] = y_hat_df  

    # compuate mse
    reg_mse = mean_squared_error(y,y_hat)
    # compute residual value of y, y-y_hat, the residual value is the y in next round of loop
    y_residual = y - y_hat
    data["{}res{}".format(y_name,x_name)] = y_residual

    # print mse, r^2, variance
    print("the mse is")
    print(reg_mse)
    print("regression score is")
#     print(r2_score(data['{}'.format(y_name)].iloc[3:], data['predicted_{}'.format(y_name)].iloc[3:]))
    # score is the r2_score, same results
    print(reg_y.score(X,y))
    r2 = reg_y.score(X,y)
#     print("var_reg is")
  #   print(df['predicted_{}'.format(y_name)].var(ddof=0))
#     var_reg = df['predicted_{}'.format(y_name)].var(ddof=0)  

    #print explained_variance_score
    print("explained_variance_score")
    variance_score = explained_variance_score(y,y_hat)
    print(variance_score)

    return reg_mse,reg_y.score(X,y),variance_score,r2


def boosting(x_list, y_name, maxlag):
  
    # loop through each variable in the list
    temp_y_name = y_name
    mse_arr = []
    r2_arr = []
    
    predicted_name_list = []
    
    for pivot_x in range(0,len(x_list)):
        print("=========this is regression round {}=========".format(pivot_x+1))

        # save return value of regression in res_list
        res_list = regression(df, x_list[pivot_x],y_name,3)  
        
        # save predicted column name as a list
        predicted_name_list.append('predicted_{}'.format(y_name))
        
        # build y_name such as x1resx1, which means x1 substacts x1_hat, res means residual
        y_name = str(y_name) +"res"+ str(x_list[pivot_x])
        
        # example: [0.7614110755692759, 0.6019695603895466, 0.4941602516989991, 0.36284165024184334]
        mse_arr.append(res_list[0])
        r2_arr.append(res_list[3])
    
    return mse_arr,predicted_name_list,r2_arr,maxlag


def causality_test(boosting_result_list):
    
    mse_arr = boosting_result_list[0]
    name_list = boosting_result_list[1]
    r2_arr =  boosting_result_list[2]
    maxlag = boosting_result_list[3]
    
    print('------------Causalilty Test Criterias------------')
    
    # mse_y means the mse to predict y using all other varaibles except for the causing variable

    mse_y = mse_arr[len(mse_arr)-2]
#     print(mse_arr[len(mse_arr)-1])
    mse_all = mse_arr[len(mse_arr)-1]

    print("mse before adding causing variable is ")
    print(mse_y)
    print("mse of all variables is")
    print(mse_all)
    print("\n!!!!!!!!!!!!!!!!!!!!!!!")
    print("change of mse -> np.log(mse_change)")
    mse_change = mse_y/mse_all
#     mse_change = ((mse_y-mse_all)/(3-2))/(mse_all/(999-3))
    
    print(np.log(mse_change))
    print("!!!!!!!!!!!!!!!!!!!!!!!\n")
    
    
    print("~~~~~~~~~~~~~~~~~")
    print("the F-score is")
    f_score = ((mse_y-mse_all)/mse_all)*((n-k*maxlag)/maxlag)
    print(n-k*maxlag)
    print(maxlag)
    print(k*maxlag)
    print(f_score)
    p_value = scipy.stats.f.sf(f_score, maxlag, n-k*maxlag)
    print("the p_value is")
    print(p_value)
    print("~~~~~~~~~~~~~~~~~")
    
    
    df['pred_y'] = df[name_list[0]]
    for key in range(1, len(name_list)):
        df['pred_y'] += df[name_list[key]]

    df['last_step'] = df['pred_y'] - df[name_list[len(name_list)-1]]

    # df['step_3'] = df['predicted_x3'] + df['predicted_x3resx3'] + df['predicted_x3resx3resx4']

    r2_y = r2_arr[len(r2_arr)-2]
#     print(mse_arr[len(mse_arr)-1])
    r2_all = r2_arr[len(r2_arr)-1]
  
    print("r_square_last is")
    print(r2_y)
    print("r_square_final is ")
    print(r2_all)
    print("\n!!!!!!!!!!!!!!!!!!!!!!!")
    print("r-square change")
    r_square_change = abs(r2_all-r2_y)/r2_y
    print(r_square_change)
    print("!!!!!!!!!!!!!!!!!!!!!!!\n")
    
    
#     # draw graph if var_change >0.05  -- to do
#     if var_change > 0.05:
#         g.edge(,,label = " {} ".format(temp_lag))

    return name_list



shoud exist: x1→x3

In [0]:
causality_test(boosting(["x3","x4","x2","x1"], "x3", 3))


=========this is regression round 1=========
the mse is
2.1734065535202913
regression score is
0.6125020174009941
explained_variance_score
0.6125020174009941
=========this is regression round 2=========
the mse is
2.1593844062611676
regression score is
0.0064516908888545554
explained_variance_score
0.0064516908888545554
=========this is regression round 3=========
the mse is
1.8986410956216886
regression score is
0.12074890875540722
explained_variance_score
0.12074890875540722
=========this is regression round 4=========
the mse is
1.3893607050990195
regression score is
0.26823415531091244
explained_variance_score
0.26823415531091244
------------Causalilty Test Criterias------------
mse before adding causing variable is 
1.8986410956216886
mse of all variables is
1.3893607050990195

!!!!!!!!!!!!!!!!!!!!!!!
change of mse -> np.log(mse_change)
0.31229470049924835
!!!!!!!!!!!!!!!!!!!!!!!

~~~~~~~~~~~~~~~~~
the F-score is
987
3
12
120.5973710549246
the p_value is
1.5223940229239123e-66
~~~

['predicted_x3',
 'predicted_x3resx3',
 'predicted_x3resx3resx4',
 'predicted_x3resx3resx4resx2']

shouldn't exist: x3→x1

In [0]:
causality_test(boosting(["x1","x4","x2","x3"], "x1", 3))


=========this is regression round 1=========
the mse is
1.0027648849710222
regression score is
0.8991914094015558
explained_variance_score
0.899191409401556
=========this is regression round 2=========
the mse is
1.0009588237947507
regression score is
0.0018010813933955205
explained_variance_score
0.0018010813933954095
=========this is regression round 3=========
the mse is
0.9977630608092803
regression score is
0.0031927017470656516
explained_variance_score
0.0031927017470656516
=========this is regression round 4=========
the mse is
0.9951017652357406
regression score is
0.0026672620766109434
explained_variance_score
0.0026672620766108324
------------Causalilty Test Criterias------------
mse before adding causing variable is 
0.9977630608092803
mse of all variables is
0.9951017652357406

!!!!!!!!!!!!!!!!!!!!!!!
change of mse -> np.log(mse_change)
0.0026708255580065493
!!!!!!!!!!!!!!!!!!!!!!!

~~~~~~~~~~~~~~~~~
the F-score is
987
3
12
0.8798760833141169
the p_value is
0.45096371300371

['predicted_x1',
 'predicted_x1resx1',
 'predicted_x1resx1resx4',
 'predicted_x1resx1resx4resx2']

shoud not exist: x4→x1

In [0]:
causality_test(boosting(["x4","x3","x2","x1"], "x4", 3))

=========this is regression round 1=========
the mse is
1.4933491125802614
regression score is
0.9026368010369424
explained_variance_score
0.9026368010369424
=========this is regression round 2=========
the mse is
1.0190131800383022
regression score is
0.3176323128637918
explained_variance_score
0.3176323128637917
=========this is regression round 3=========
the mse is
1.016692066935972
regression score is
0.0022778047897701326
explained_variance_score
0.0022778047897701326
=========this is regression round 4=========
the mse is
1.0158701253737694
regression score is
0.0008084469122295879
explained_variance_score
0.0008084469122294768
------------Causalilty Test Criterias------------
mse before adding causing variable is 
1.016692066935972
mse of all variables is
1.0158701253737694

!!!!!!!!!!!!!!!!!!!!!!!
change of mse -> np.log(mse_change)
0.0008087738816713564
!!!!!!!!!!!!!!!!!!!!!!!

~~~~~~~~~~~~~~~~~
the F-score is
987
3
12
0.26619423803329545
the p_value is
0.8497865577880883
~~~

['predicted_x4',
 'predicted_x4resx4',
 'predicted_x4resx4resx3',
 'predicted_x4resx4resx3resx2']

should exist x3→x4

In [0]:
causality_test(boosting(["x4","x1","x2","x3"], "x4", 3))

=========this is regression round 1=========
the mse is
1.4949793456809346
regression score is
0.9029283078383323
explained_variance_score
0.9029283078383323
=========this is regression round 2=========
the mse is
1.1710344867473235
regression score is
0.21668851805177317
explained_variance_score
0.21668851805177314
=========this is regression round 3=========
the mse is
1.1406118754690402
regression score is
0.025979261603802373
explained_variance_score
0.025979261603802484
=========this is regression round 4=========
the mse is
1.0972610834166792
regression score is
0.03800661117484361
explained_variance_score
0.03800661117484372
------------Causalilty Test Criterias------------
mse before adding causing variable is 
1.1406118754690402
mse of all variables is
1.0972610834166792

!!!!!!!!!!!!!!!!!!!!!!!
change of mse -> np.log(mse_change)
0.03874770066316748
!!!!!!!!!!!!!!!!!!!!!!!

~~~~~~~~~~~~~~~~~
the F-score is
987
3
12
12.998192317926858
the p_value is
2.4859939343987365e-08
~~~~

['predicted_x4',
 'predicted_x4resx4',
 'predicted_x4resx4resx1',
 'predicted_x4resx4resx1resx2']

In [0]:
# df = pd.read_csv("synth_data_with_header.csv", header='infer')
# df['x1_1'] = df['x1'].shift(1)

# df

# # regression() 

df = df.iloc[3:]
# df

X = df[['x1_1','x2_1','x3_1','x4_1','x1_2','x2_2','x3_2','x4_2','x1_3','x2_3','x3_3','x4_3']]
y = df['x4']
maxlag=3

data = df
y_name = 'x4'

# build regression reg_y, X→y
reg_y = LinearRegression()
# fit model using data X, y
reg_y.fit(X, y)
# check number of features
#     print(reg_y.n_features_)
# check feature importance 
#     print(reg_y.feature_importances_)

# y_hat is the predicted value of y
y_hat = reg_y.predict(X) 

# save predicted y_hat as a pd dataframe and move its index to match the place in original df
y_hat_df = pd.DataFrame(y_hat)
y_hat_df.index += maxlag
# save the predicted value into dataframe

# compuate mse
reg_mse = mean_squared_error(y,y_hat)
# compute residual value of y, y-y_hat, the residual value is the y in next round of loop
y_residual = y - y_hat

# print mse, r^2, variance
print("the mse is")
print(reg_mse)
print("regression score is")
#     print(r2_score(data['{}'.format(y_name)].iloc[3:], data['predicted_{}'.format(y_name)].iloc[3:]))
# score is the r2_score, same results
print(reg_y.score(X,y))

the mse is
0.9353482940775073
regression score is
0.9392661564663444
